In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from preprocess import load_train_test_split
from model import create_mlp_model, compile_model, get_model_summary
from train import train_model, evaluate_model, save_model
from config import MODEL_CONFIG, TRAIN_CONFIG

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Load Preprocessed Data

In [ ]:
# Load the train/test splits
splits = load_train_test_split()

X_train = splits['X_train']
X_test = splits['X_test']
y_train = splits['y_train']
y_test = splits['y_test']

print(f"Training set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nFeature names: {list(X_train.columns)}")

In [ ]:
print("Model configuration:")
print(MODEL_CONFIG)

In [ ]:
# Create the model
model = create_mlp_model(input_shape=X_train.shape[1])

# Compile the model
model = compile_model(model)

# Display model summary
get_model_summary(model)

In [ ]:
print("Training configuration:")
print(TRAIN_CONFIG)

In [ ]:
# Train the model
model, history = train_model(
    X_train, y_train,
    model=model,
    verbose=1  # Show training progress
)

## Training History Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot loss
axes[0].plot(history.history['loss'], label='Training Loss', marker='o')
axes[0].plot(history.history['val_loss'], label='Validation Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Model Loss During Training')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', marker='o')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Model Accuracy During Training')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Save the Trained Model

In [ ]:
save_model(model)